Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать `pint` + `uncertainties`.

In [128]:
from pint import UnitRegistry
from uncertainties import ufloat, umath
from scipy.constants import k as k_J_per_K


In [129]:
H_отсчета = (ufloat(200, 1) * ureg.kilometer).to_base_units()
H_отсчета

<Quantity(200000+/-1000, 'meter')>

In [130]:
P_отсчета =  (ufloat(8, 1)* 1e-5 * ureg.pascal).to_base_units()
P_отсчета

<Quantity((8.00000+/-1.00000)e-05, 'kilogram / meter / second ** 2')>

In [131]:
H = (ufloat(60, 10) * ureg.kilometer).to_base_units()
H

<Quantity(60000.0+/-10000.0, 'meter')>

In [132]:
σ = (ufloat(8, 2) * ureg.angstrom * ureg.angstrom).to_base_units()
σ

<Quantity((8.00000+/-2.00000)e-20, 'meter ** 2')>

In [133]:
T_отсчета = (ufloat(854.4, 200) * ureg.K).to_base_units()
T_отсчета

<Quantity(854.400+/-200.000, 'kelvin')>

In [134]:
T_экзосферы = (ufloat(1000, 300) * ureg.kelvin).to_base_units()
T_экзосферы

<Quantity(2000.00+/-200.00, 'kelvin')>

Todo: разобраться, почему `ureg` не может сам разобраться с конверсией единиц измерения

In [135]:
k = (k_J_per_K * ureg.J / ureg.K).to_base_units()
k

<Quantity(1.380649e-23, 'kilogram * meter ** 2 / second ** 2 / kelvin')>

In [136]:
p_экз = k * T_экзосферы /  σ / H
p_экз.to('pascal')

<Quantity((5.75270+/-1.82169)e-06, 'pascal')>

In [137]:
h_экз = H_отсчета + H * umath.log( (P_отсчета / p_экз).magnitude )
h_экз.to('km')

<Quantity(357.941+/-40.468, 'kilometer')>